In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:


class CustomDecisionTreeClassifier:
    def __init__(self, max_depth: int = 4, min_samples_split: int = 2, min_information_gain: float = 1e-4):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_information_gain = min_information_gain
        self.tree = None
        self.feature_names = None  # Store feature names
    
    def _entropy(self, p: float) -> float:
        if (p == 0) or (p == 1):
            return 0
        
        return -p * np.log2(p) - (1 - p) * np.log2(1 - p)
    
    def _weighted_entropy(self, y: np.ndarray, left_indices: np.ndarray, right_indices: np.ndarray) -> float:
        w_left = len(left_indices) / len(y)
        w_right = len(right_indices) / len(y)
        p_left = np.sum(y[left_indices]) / len(left_indices) if len(left_indices) > 0 else 0
        p_right = np.sum(y[right_indices]) / len(right_indices) if len(right_indices) > 0 else 0
        
        return w_left * self._entropy(p_left) + w_right * self._entropy(p_right)
    
    def _information_gain(self, y: np.ndarray, left_indices: np.ndarray, right_indices: np.ndarray) -> float:
        p_node = np.sum(y) / len(y)
        h_node = self._entropy(p_node)
        return h_node - self._weighted_entropy(y, left_indices, right_indices)
    
    def _best_split(self, X: np.ndarray, y: np.ndarray) -> tuple[int, float, np.ndarray, np.ndarray, float]:
        best_feature, best_threshold, best_info_gain = None, None, -float('inf')
        best_left_indices, best_right_indices = None, None
        
        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = np.where(X[:, feature] <= threshold)[0]
                right_indices = np.where(X[:, feature] > threshold)[0]
                
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                
                info_gain = self._information_gain(y, left_indices, right_indices)
                
                if info_gain > best_info_gain:
                    best_feature, best_threshold, best_info_gain = feature, threshold, info_gain
                    best_left_indices, best_right_indices = left_indices, right_indices
                    
        return best_feature, best_threshold, best_left_indices, best_right_indices, best_info_gain
    
    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> dict | int:
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))
        
        if num_classes == 1 or depth >= self.max_depth or num_samples < self.min_samples_split:
            return np.argmax(np.bincount(y))  # Return the most common class
        
        feature, threshold, left_indices, right_indices, info_gain = self._best_split(X, y)
        
        if feature is None or info_gain < self.min_information_gain:
            return np.argmax(np.bincount(y))  # Stop if no gain
        
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        
        return {
            'feature': self.feature_names[feature] if self.feature_names else feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }
    
    def fit(self, X: pd.DataFrame, y: pd.Series) -> None:
        self.feature_names = X.columns.tolist()  # Store feature names
        X = np.array(X)
        y = np.array(y)
        self.tree = self._build_tree(X, y)
    
    def _predict_sample(self, node: dict | int, x: np.ndarray) -> int:
        if isinstance(node, dict):
            feature_idx = self.feature_names.index(node['feature']) if isinstance(node['feature'], str) else node['feature']
            if x[feature_idx] <= node['threshold']:
                return self._predict_sample(node['left'], x)
            else:
                return self._predict_sample(node['right'], x)
        return node
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        return np.array([self._predict_sample(self.tree, x) for x in np.array(X)])